# Docker Compose

## Docker Compose: la herramienta todo en uno

Todo lo que hemos hecho hasta ahora esta bien, pero es un poco tedioso. Para ello esta docker compose.

Si estas usando Windows esta instalada por defecto con docker desktop. Si estas usando Linux debes instalarla. Parece que ya la tengo. 